In [1]:
#Checking for Dataset
import os
print(os.getcwd())
if len(os.listdir('Dataset/'))==0:
    print("Dataset not available...")
else:
    os.chdir("Dataset/")
    print(os.listdir())

E:\Projects\Pubg-Competition
['sample_submission_V2.csv', 'test_V2.csv', 'train_V2.csv']


In [2]:
#Importing Dependencies
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.preprocessing import normalize 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [3]:
#Loading Data
data_df=pd.read_csv('train_V2.csv')
test_data_df=pd.read_csv('test_V2.csv')

In [4]:
data_df.describe(include=np.number).drop('count').T

,mean,std,min,25%,50%,75%,max
assists,0.233815,0.588573,0.0,0.0,0.0000,0.000000,22.0
boosts,1.106908,1.715794,0.0,0.0,0.0000,2.000000,33.0
damageDealt,130.717138,170.780621,0.0,0.0,84.2400,186.000000,6616.0
DBNOs,0.657876,1.145743,0.0,0.0,0.0000,1.000000,53.0
headshotKills,0.226820,0.602155,0.0,0.0,0.0000,0.000000,64.0
heals,1.370147,2.679982,0.0,0.0,0.0000,2.000000,80.0
killPlace,47.599350,27.462937,1.0,24.0,47.0000,71.000000,101.0
killPoints,505.006042,627.504896,0.0,0.0,0.0000,1172.000000,2170.0
kills,0.924783,1.558445,0.0,0.0,0.0000,1.000000,72.0
killStreaks,0.543955,0.710972,0.0,0.0,0.0000,1.000000,20.0


In [5]:
#Filling NaN with 0
data_df=data_df.fillna(0)
data_df.isna().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       0
dtype: int64

In [6]:
#Totalling the distance
data_df['totalDistance']=data_df['rideDistance']+data_df['walkDistance']+data_df['swimDistance']
test_data_df['totalDistance']=test_data_df['rideDistance']+test_data_df['walkDistance']+test_data_df['swimDistance']

data_df=data_df.drop(columns=['walkDistance','swimDistance','rideDistance'])
test_data_df=test_data_df.drop(columns=['walkDistance','swimDistance','rideDistance'])

In [7]:
data_df=data_df.drop(columns=['longestKill','Id','groupId','matchId','killPoints','matchDuration','rankPoints','winPoints'])
test_data_df=test_data_df.drop(columns=['longestKill','Id','groupId','matchId','killPoints','matchDuration','rankPoints','winPoints'])

In [8]:
'''
#converting matchType to oneHot
one_hot = pd.get_dummies(data_df['matchType'], drop_first=True)
data_df=data_df.join(one_hot)

one_hot = pd.get_dummies(test_data_df['matchType'], drop_first=True)
test_data_df=test_data_df.join(one_hot)
'''

"\n#converting matchType to oneHot\none_hot = pd.get_dummies(data_df['matchType'], drop_first=True)\ndata_df=data_df.join(one_hot)\n\none_hot = pd.get_dummies(test_data_df['matchType'], drop_first=True)\ntest_data_df=test_data_df.join(one_hot)\n"

In [9]:
data_df=data_df.drop(columns=['matchType','boosts','killStreaks','roadKills','teamKills','weaponsAcquired','numGroups','vehicleDestroys'])
test_data_df=test_data_df.drop(columns=['matchType','boosts','killStreaks','roadKills','teamKills','weaponsAcquired','numGroups','vehicleDestroys'])

In [10]:
data_df.head()

,assists,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,maxPlace,revives,winPlacePerc,totalDistance
0,0,0.00,0,0,0,60,0,28,0,0.4444,244.8000
1,0,91.47,0,0,0,57,0,26,0,0.6400,1445.0445
2,1,68.00,0,0,0,47,0,50,0,0.7755,161.8000
3,0,32.90,0,0,0,75,0,31,0,0.1667,202.7000
4,0,100.00,0,0,0,45,1,97,0,0.1875,49.7500


In [11]:
test_data_df.head()

,assists,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,maxPlace,revives,totalDistance
0,0,51.46,0,0,0,73,0,28,0,588.0
1,0,179.10,0,0,2,11,2,48,2,6686.0
2,1,23.40,0,0,4,49,0,28,0,787.8
3,0,65.52,0,0,0,54,0,45,0,1812.0
4,0,330.20,1,2,1,7,3,28,1,2963.0


In [12]:
#Converting Dataframes to numpy array
y_train=data_df['winPlacePerc'].ravel()
train=data_df.drop(['winPlacePerc'], axis=1)
X_train=train.values
X_test=test_data_df.values
print(X_train.shape,"X_train")
print(y_train.shape,"y_train")
print(X_test.shape,"X_test")

(4446966, 10) X_train
(4446966,) y_train
(1934174, 10) X_test


In [13]:
#Normalizing the Data
X_test = normalize(X_test, axis=0, norm='max')
X_train = normalize(X_train, axis=0, norm='max')

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [19]:
model1 = XGBRegressor(n_estimators=1000, max_depth=17, learning_rate=0.02, objective='binary:logistic', silent=True)

In [20]:
model1.fit(X_train, y_train, eval_set=[[X_val, y_val]], eval_metric='mae', early_stopping_rounds=10, verbose=True)

[0]	validation_0-mae:0.264326
Will train until validation_0-mae hasn't improved in 10 rounds.
[1]	validation_0-mae:0.259519
[2]	validation_0-mae:0.254812
[3]	validation_0-mae:0.250207
[4]	validation_0-mae:0.245703
[5]	validation_0-mae:0.241298
[6]	validation_0-mae:0.236991
[7]	validation_0-mae:0.232778
[8]	validation_0-mae:0.228661
[9]	validation_0-mae:0.224636
[10]	validation_0-mae:0.220701
[11]	validation_0-mae:0.216855
[12]	validation_0-mae:0.213096
[13]	validation_0-mae:0.209423
[14]	validation_0-mae:0.205831
[15]	validation_0-mae:0.202322
[16]	validation_0-mae:0.198893
[17]	validation_0-mae:0.195544
[18]	validation_0-mae:0.192271
[19]	validation_0-mae:0.189074
[20]	validation_0-mae:0.18595
[21]	validation_0-mae:0.1829
[22]	validation_0-mae:0.179921
[23]	validation_0-mae:0.177012
[24]	validation_0-mae:0.174171
[25]	validation_0-mae:0.171398
[26]	validation_0-mae:0.168689
[27]	validation_0-mae:0.166043
[28]	validation_0-mae:0.163459
[29]	validation_0-mae:0.160938
[30]	validation_0-m

KeyboardInterrupt: 

In [ ]:
#Predicting and Creating the Submission
sub=pd.read_csv('test_V2.csv', index_col='Id')
predictions = model1.predict(X_test).reshape(-1,1)
dfpredictions = pd.DataFrame(predictions, index=sub.index).rename(columns={0:'winPlacePerc'})
dfpredictions.to_csv('submission.csv', header=True)